In [1]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
)
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
import os

In [2]:
# Load your CoNLL-format data
def load_conll_data(filepath):
    tokens, labels = [], []
    with open(filepath, encoding='utf-8') as f:
        words, tags = [], []
        for line in f:
            line = line.strip()
            if not line:
                if words:
                    tokens.append(words)
                    labels.append(tags)
                    words, tags = [], []
            else:
                splits = line.split()
                if len(splits) == 2:
                    words.append(splits[0])
                    tags.append(splits[1])
    return Dataset.from_dict({"tokens": tokens, "ner_tags": labels})

raw_dataset = load_conll_data("../data/labeled/ner_sample.conll")

dataset = DatasetDict({
    "train": raw_dataset.train_test_split(test_size=0.2, seed=42)["train"],
    "test": raw_dataset.train_test_split(test_size=0.2, seed=42)["test"],
})

In [3]:

# === Tokenizer & Model ===
model_checkpoint = "rasyosef/bert-tiny-amharic"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Label mapping
label_list = sorted(set(tag for tags in dataset["train"]["ner_tags"] for tag in tags))
label_to_id = {l: i for i, l in enumerate(label_list)}
id_to_label = {i: l for l, i in label_to_id.items()}


# Tokenization with label alignment
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []

    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label_to_id[label[word_idx]])
            else:
                label_ids.append(label_to_id[label[word_idx]])
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [4]:

# === Model ===
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label=id_to_label,
    label2id=label_to_id,
)

# === Training Arguments ===
training_args = TrainingArguments(
    output_dir="./ner_model_bert_tiny_amharic",
    do_eval=True,                      # ← enables evaluation
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs_bert_tiny",
    save_total_limit=2,
)


# === Data Collator & Metric Function ===
data_collator = DataCollatorForTokenClassification(tokenizer)
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[id_to_label[l] for l in label if l != -100] for label in labels]
    pred_labels = [[id_to_label[p] for (p, l) in zip(pred, label) if l != -100]
                   for pred, label in zip(predictions, labels)]

    precision, recall, f1, _ = precision_recall_fscore_support(
        [item for sublist in true_labels for item in sublist],
        [item for sublist in pred_labels for item in sublist],
        average="macro"
    )

    return {"precision": precision, "recall": recall, "f1": f1}

Some weights of BertForTokenClassification were not initialized from the model checkpoint at rasyosef/bert-tiny-amharic and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:

# === Trainer ===
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train and Evaluate
trainer.train()


/var/folders/pp/3nx806hs7bl388g7c6clvc5h0000gn/T/ipykernel_22666/2544475920.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/Users/oza/Developer/10 Acadamy/week4/amharic-ecommerce-ner/venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


TrainOutput(global_step=3, training_loss=0.0, metrics={'train_runtime': 1.1749, 'train_samples_per_second': 7.66, 'train_steps_per_second': 2.553, 'total_flos': 5122765674.0, 'train_loss': 0.0, 'epoch': 3.0})

In [6]:
metrics = trainer.evaluate()
print(metrics)

# Save metrics to file
import json
with open("metrics_bert_tiny.json", "w", encoding="utf-8") as f:
    json.dump(metrics, f, indent=2, ensure_ascii=False)


/Users/oza/Developer/10 Acadamy/week4/amharic-ecommerce-ner/venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_runtime': 0.1102, 'eval_samples_per_second': 9.075, 'eval_steps_per_second': 9.075, 'epoch': 3.0}
